# ERC-4626: list all vaults

This notebook serves both as a coding tutorial and a useful data analytics tool for ERC-4626 vaults. 

- In this notebook, we examine different ERC-4626 vaults across different EVM blockchains   
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- Read general instructions [how to run the tutorials](./)
- See `ERC-4626 scanning all vaults onchain` example in tutorials first how to build a vault database as local `vault_db.pickle` file.




## Setup

- Set up notebook renderinb parmaeters

In [13]:
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [14]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 8067 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [15]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")

df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())

DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Denomination, NAV, Protocol, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


Symbol           Name  \
Chain    Address                                                               
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a   UB-WETH        UB-WETH   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701      sDAI   Savings xDAI   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  yvUSDC-A  USDC yVault-A   
         0xbb287e6017d3deb0e2e65061e8684eab21060123  yvUSDT-A  USDT yVault-A   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952  bathUSDC   BathToken v1   

                                                    Denomination  \
Chain    Address                                                   
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a         WETH   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701        WXDAI   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0         USDC   
         0xbb287e6017d3deb0e2e65061e8684eab21060123         USDT   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952         USDC   

                                                              NAV  \
Chain    Address                                                    
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a        406.41   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701 72,354,403.83   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0    568,182.95   
         0xbb287e6017d3deb0e2e65061e8684eab21060123    439,995.87   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952     60,605.57   

                                                               Protocol  \
Chain    Address                                                          
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a  <unknown ERC-4626>   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701  <unknown ERC-4626>   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0            Yearn v3   
         0xbb287e6017d3deb0e2e65061e8684eab21060123            Yearn v3   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952  <unknown ERC-4626>   

                                                      Mgmt fee   Perf fee  \
Chain    Address                                                            
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a  <unknown>  <unknown>   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701  <unknown>  <unknown>   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0  <unknown>  <unknown>   
         0xbb287e6017d3deb0e2e65061e8684eab21060123  <unknown>  <unknown>   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952  <unknown>  <unknown>   

                                                                          Shares  \
Chain    Address                                                                   
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a        404.83166035272080632   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701  61664855.443303215024358097   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0                481865.811675   
         0xbb287e6017d3deb0e2e65061e8684eab21060123                370774.866903   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952              2.7245664273E-8   

                                                             First seen  \
Chain    Address                                                          
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a 2025-01-13 06:53:05   
Gnosis   0xaf204776c7245bf4147c2612bf6e5972ee483701 2023-09-28 12:13:30   
Polygon  0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0 2023-10-26 16:45:48   
         0xbb287e6017d3deb0e2e65061e8684eab21060123 2023-12-07 17:02:20   
Optimism 0xe0e112e8f33d3f437d1f895cbb1a456836125952 2022-05-24 23:22:04   

                                                                                       _detection_data  \
Chain    Address                                                                                         
Soneium  0x232554b4b291a446b4829300bec133fbb07a8f2a  ERC4262VaultDetection(chain=1868, address='0x2...   
Gnosis   0xaf204776c7245bf4147c261

## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [16]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True
erc_7575 = df["ERC-7575"] == True
fee_detected = df["Fee detected"] == True

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame(
    {
        "Total vaults detected": df.groupby(level="Chain").size(),
        "Total vault TVL/NAV in stable vaults": stablecoin_only_df.groupby(level="Chain")["NAV"].sum(),
        "Protocol correctly identified": df[identified_filter].groupby(level="Chain").size().astype(int),
        "Stablecoin denominated": df[stablecoin_denominated].groupby(level="Chain").size().astype(int),
        f"Notable stablecoin NAV (min {nav_threshold} USD)": df[notable_nav].groupby(level="Chain").size().astype(int),
        f"ERC-7540": df[erc_7540].groupby(level="Chain").size().astype(int),
        f"ERC-7575": df[erc_7575].groupby(level="Chain").size().astype(int),
        f"Fee data supported": df[fee_detected].groupby(level="Chain").size().astype(int),
    }
).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,1931,"163,423,141.20",345.00,669,58.00,19.00,27.00,220.00
Avalanche,299,"157,780,442.63",43.00,87,10.00,1.00,1.00,48.00
Base,1209,"304,088,417.27",598.00,459,54.00,35.00,15.00,97.00
Berachain,302,"179,444,521.94",38.00,19,12.00,0.00,0.00,2.00
Binance,345,"254,947,613.04",43.00,99,19.00,0.00,0.00,25.00
Blast,106,"2,001,517.67",2.00,14,2.00,0.00,0.00,32.00
Celo,5,"3,191,558.97",0.00,2,1.00,0.00,0.00,0.00
Ethereum,2161,"9,138,688,833.19",544.00,745,200.00,18.00,16.00,187.00
Gnosis,87,"74,837,573.08",12.00,35,5.00,0.00,0.00,0.00


## Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [17]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True
erc_7575 = df["ERC-7575"] == True
fee_detected = df["Fee detected"] == True

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame(
    {
        "Total vaults detected": df.groupby("Protocol").size(),
        "Total vault TVL/NAV in stable vaults": stablecoin_only_df.groupby("Protocol")["NAV"].sum(),
        "Protocol correctly identified": df[identified_filter].groupby("Protocol").size().astype(int),
        "Stablecoin denominated": df[stablecoin_denominated].groupby("Protocol").size().astype(int),
        f"Notable stablecoin NAV (min {nav_threshold} USD)": df[notable_nav].groupby("Protocol").size().astype(int),
        f"ERC-7540": df[erc_7540].groupby("Protocol").size().astype(int),
        f"ERC-7575": df[erc_7575].groupby("Protocol").size().astype(int),
        f"Fee data supported": df[fee_detected].groupby("Protocol").size().astype(int),
    }
).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<unknown ERC-4626>,5758,"8,140,001,224.39",0.00,"2,065.00",230.00,0.00,12.00,0.00
Morpho,196,"1,324,185,710.78",196.00,111.00,64.00,0.00,0.00,196.00
Fluid,59,"428,340,233.10",59.00,26.00,8.00,0.00,0.00,0.00
Euler Vault Kit,305,"298,503,468.10",305.00,91.00,35.00,0.00,0.00,0.00
Yearn v3,129,"77,400,550.15",129.00,65.00,17.00,0.00,0.00,0.00
Superform,136,"26,502,641.00",136.00,55.00,10.00,19.00,0.00,0.00
Kiln Metavault,72,"18,808,215.43",72.00,63.00,8.00,0.00,0.00,0.00
Yearn compounder,71,"17,444,762.32",71.00,12.00,1.00,0.00,0.00,0.00
Term Finance,16,"10,291,528.41",16.00,15.00,5.00,0.00,0.00,0.00


## Vault deployment history

- Show how much history we have for each chain


In [18]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level="Chain")["First seen"].agg(["min", "max"]).reset_index()

# Rename columns for clarity
seen_df.columns = ["Chain", "First vault deployed", "Last vault deployed"]

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-04-17 22:59:29
Avalanche,2022-04-11 00:05:24,2025-04-14 20:36:59
Base,2023-08-04 15:36:07,2025-04-21 12:51:07
Berachain,2025-01-26 00:06:03,2025-04-20 13:52:05
Binance,2022-05-27 17:25:18,2025-04-18 22:25:52
Blast,2024-02-29 20:15:37,2024-12-03 15:38:15
Celo,2022-08-08 21:14:10,2025-01-29 23:13:18
Ethereum,2019-06-11 06:17:19,2025-04-20 21:45:11
Gnosis,2022-05-01 18:47:35,2025-04-20 17:09:50


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [19]:
largest_threshold = 20
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100]
largest_df = largest_df[largest_df["Stablecoin denominated"] == True]
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))

,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Savings USDS,"2,538,358,053.64",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,29977,<unknown>,<unknown>
Staked USDe,"2,108,822,518.82",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,69739,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"901,623,132.31",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,54489,<unknown>,<unknown>
Savings Dai,"499,780,359.53",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,73251,<unknown>,<unknown>
Syrup USDC,"223,832,571.56",Ethereum,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,USDC,4662,<unknown>,<unknown>
Staked USDX,"199,881,840.63",Binance,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,USDX,5532,<unknown>,<unknown>
Usual Boosted USDC,"186,343,422.05",Ethereum,0xd63070114470f685b75b74d60eec7c1113d33a3d,USDC,18036,0.00,0.00
Fluid USD Coin,"178,682,432.58",Ethereum,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,USDC,18117,<unknown>,<unknown>
Bridged USDC (Stargate)Vault,"170,659,530.48",Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,171993,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [20]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby("Chain")["NAV"].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

NAV  \
Chain       Name                                            
Arbitrum    ostiumLP                        53,877,262.30   
Avalanche   EVK Vault eUSDC-2               54,238,946.06   
Base        Spark USDC Vault                87,951,820.45   
            Spark USDC Vault                 4,603,481.57   
Berachain   Bridged USDC (Stargate)Vault   170,659,530.48   
Binance     Staked USDX                    199,881,840.63   
Blast       Wasabi USDB Vault                1,499,179.72   
Celo        Staked EURA                      3,191,557.90   
Ethereum    Savings USDS                 2,538,358,053.64   
Gnosis      Savings xDAI                    72,354,403.83   
Hyperliquid hyUSDXL (Looped HYPE) - 1        1,132,890.11   
Mode        USDC Ironclad Vault                337,647.27   
Optimism    exactly USDC                     3,787,638.12   
Polygon     Compound USDC                   21,925,074.22   
Soneium     UB-USDC                          1,194,943.97   
ZKsync      hsUSDC                             723,549.95   

                                                                             Address  \
Chain       Name                                                                       
Arbitrum    ostiumLP                      0x20d419a8e12c45f88fda7c5760bb6923cee27f98   
Avalanche   EVK Vault eUSDC-2             0x39de0f00189306062d79edec6dca5bb6bfd108f9   
Base        Spark USDC Vault              0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a   
            Spark USDC Vault              0x3128a0f7f0ea68e7b7c9b00afa7e41045828e858   
Berachain   Bridged USDC (Stargate)Vault  0x90bc07408f5b5eac4de38af76ea6069e1fcee363   
Binance     Staked USDX                   0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92   
Blast       Wasabi USDB Vault             0x4bed2a922654cacc2be974689619768fabf24855   
Celo        Staked EURA                   0x004626a008b1acdc4c74ab51644093b155e59a23   
Ethereum    Savings USDS                  0xa3931d71877c0e7a3148cb7eb4463524fec27fbd   
Gnosis      Savings xDAI                  0xaf204776c7245bf4147c2612bf6e5972ee483701   
Hyperliquid hyUSDXL (Looped HYPE) - 1     0xaeedd5b6d42e0f077ccf3e7a78ff70b8cb217329   
Mode        USDC Ironclad Vault           0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c   
Optimism    exactly USDC                  0x6926b434cce9b5b7966ae1bfeef6d0a7dcf3a8bb   
Polygon     Compound USDC                 0x781fb7f6d845e3be129289833b04d43aa8558c42   
Soneium     UB-USDC                       0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41   
ZKsync      hsUSDC                        0xaf08a9d918f16332f22cf8dc9abe9d9e14ddcbc2   

                                         Denomination  Total events  \
Chain       Name                                                      
Arbitrum    ostiumLP                             USDC          4391   
Avalanche   EVK Vault eUSDC-2                    USDC          1906   
Base        Spark USDC Vault                     USDC          9773   
            Spark USDC Vault                     USDC         34901   
Berachain   Bridged USDC (Stargate)Vault       USDC.e        171993   
Binance     Staked USDX                          USDX          5532   
Blast       Wasabi USDB Vault                    USDB         92849   
Celo        Staked EURA                          EURA           223   
Ethereum    Savings USDS                         USDS         29977   
Gnosis      Savings xDAI                        WXDAI        562452   
Hyperliquid hyUSDXL (Looped HYPE) - 1           USDXL           385   
Mode        USDC Ironclad Vault                  USDC           183   
Optimism    exactly USDC                         USDC         61843   
Polygon     Compound USDC                        USDC          5168   
Soneium     UB-USDC                            USDC.e        229127   
ZKsync      hsUSDC                             USDC.e         48708   

                                           Mgmt fee   Perf fee  
Chain       Name          

## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [21]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
UB-WETH,727412,Soneium,0x232554b4b291a446b4829300bec133fbb07a8f2a,WETH,406.41,99 days 05:59:32.194426,535111,192301
Savings xDAI,562452,Gnosis,0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"72,354,403.83",572 days 00:39:07.194426,302148,260304
USDC yVault-A,490209,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"568,182.95",543 days 20:06:49.194426,465208,25001
USDT yVault-A,458259,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"439,995.87",501 days 19:50:17.194426,407707,50552
BathToken v1,297323,Optimism,0xe0e112e8f33d3f437d1f895cbb1a456836125952,USDC,"60,605.57",1063 days 13:30:33.194426,150209,147114
Wasabi WETH Vault,242991,Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,984.29,417 days 16:37:00.194426,161218,81773
UB-USDC,229127,Soneium,0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41,USDC.e,"1,194,943.97",99 days 06:00:26.194426,163238,65889
Beraborrow iBGT,220624,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"105,595.18",46 days 00:47:18.194426,115351,105273
UB-ASTR,210848,Soneium,0x85a4fb48c7f9383083864d62abeccdf318fd8e6f,ASTR,"49,046,750.96",99 days 05:59:56.194426,106698,104150


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [22]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby("Chain")["Total events"].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"121,981.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"970,078.39",467 days 04:20:12.194426,119876,2105
Avalanche,GoGoPool Liquid Staking Token,"70,681.00",0xa25eaf2906fa1a3a13edac9b9657108af7b703e3,WAVAX,"844,112.63",747 days 15:41:52.194426,42706,27975
Base,Moonwell Flagship ETH,"143,795.00",0xa0e430870c4604ccfc7b38ca7845b1ff653d0ff1,WETH,"12,542.84",313 days 23:55:22.194426,87060,56735
Berachain,Beraborrow iBGT,"220,624.00",0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"105,595.18",46 days 00:47:18.194426,115351,105273
Binance,kUSDT,"42,846.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"1,740,203.84",680 days 05:43:54.194426,25349,17497
Blast,Wasabi WETH Vault,"242,991.00",0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,984.29,417 days 16:37:00.194426,161218,81773
Celo,Wrapped mcUSD,241.00,0x5f71fe197fd9aad95b7b5510a98882769622112a,mcUSD,"32,043.96",949 days 17:18:40.194426,180,61
Ethereum,pufETH,"168,057.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"57,000.88",446 days 17:08:02.194426,151659,16398
Gnosis,Savings xDAI,"562,452.00",0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"72,354,403.83",572 days 00:39:07.194426,302148,260304


## Oldest vaults

- Show oldest vaults

In [23]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby("Chain")["Age"].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1060 days 21:37:31.194426,WETH,17.99,"2,336.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1104 days 19:55:15.194426,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,599 days 20:58:44.194426,WETH,0.97,"1,118.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,86 days 12:46:34.194426,USDC.e,"170,659,530.48","171,993.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1029 days 15:59:58.194426,NRCH,"1,188,908.73","1,195.00"
Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,Wasabi WETH Vault,417 days 16:37:00.194426,WETH,984.29,"242,991.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1093 days 01:27:00.194426,THOR,"80,261,173.80","19,661.00"
Gnosis,0x004626a008b1acdc4c74ab51644093b155e59a23,Staked EURA,598 days 23:10:42.194426,EURA,"614,526.09","3,602.00"
Hyperliquid,0x2831775cb5e64b1d892853893858a261e898fbeb,wHYPE,40 days 14:41:37.194426,WHYPE,"273,674.97","1,221.00"


## All vaults

- List of all stablecoin vaults

In [24]:
all_df = df.reset_index()

min_nav = 10_000

stablecoin_denominated = all_df["Stablecoin denominated"] == True
all_df = all_df[stablecoin_denominated]
all_df = all_df[all_df["NAV"] < broken_max_nav_threshold]
all_df = all_df[all_df["NAV"] >= min_nav]
all_df = all_df[["NAV", "Chain", "Protocol", "Address", "Name", "Age", "Denomination", "Total events"]]
all_df = all_df.set_index("Name").sort_values("NAV", ascending=False)

print(f"Displaying {len(all_df)} stablecoin vaults w/min NAV of {min_nav} USD")
pd.options.display.max_rows = None
display(all_df)

Displaying 588 stablecoin vaults w/min NAV of 10000 USD


,NAV,Chain,Protocol,Address,Age,Denomination,Total events
Name,,,,,,,
Savings USDS,"2,538,358,053.64",Ethereum,<unknown ERC-4626>,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,216 days 23:15:50.194426,USDS,29977
Staked USDe,"2,108,822,518.82",Ethereum,<unknown ERC-4626>,0x9d39a5de30e57443bff2a8307a4256c8797a3497,518 days 04:46:26.194426,USDe,69739
Ethereal Pre-deposit Vault,"901,623,132.31",Ethereum,<unknown ERC-4626>,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,67 days 21:38:26.194426,USDe,54489
Savings Dai,"499,780,359.53",Ethereum,<unknown ERC-4626>,0x83f20f44975d03b1b09e64809b757c47f942beea,755 days 02:27:50.194426,DAI,73251
Spark DAI Vault,"325,500,547.62",Ethereum,Morpho,0x73e65dbd630f90604062f6e02fab9138e713edd9,388 days 22:32:50.194426,DAI,97
Syrup USDC,"223,832,571.56",Ethereum,<unknown ERC-4626>,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,310 days 18:21:26.194426,USDC,4662
Staked USDX,"199,881,840.63",Binance,<unknown ERC-4626>,0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92,400 days 01:13:48.194426,USDX,5532
Usual Boosted USDC,"186,343,422.05",Ethereum,Morpho,0xd63070114470f685b75b74d60eec7c1113d33a3d,270 days 17:58:26.194426,USDC,18036
Fluid USD Coin,"178,682,432.58",Ethereum,Fluid,0x9fb7b4477576fe5b32be4c1843afb1e55f251b33,428 days 11:37:14.194426,USDC,18117
